In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from food.tools import *
from food.paths import *

import pandas as pd
import numpy as np

# !nbdev_build_lib

In [3]:
key = 'cbcff4fd4501cac6582f6e912c8f553b'